In [74]:
from dataclasses import dataclass
import heapq

# General setup code

In [75]:
class Antichain:
    def __init__(self, path: list[int]):
        self.path = path
        # self.path.sort() As written, things are sorted on generation
        self.nodes = set(path)
    
    def will_not_break(self, other):
        intersection = self.nodes & other.nodes
        result = len(intersection) < 2
        # print("---")
        # print("Call: will_not_break")
        # print(f"self: {self}")
        # print(f"other: {other}")
        # print(intersection)       
        # print(result)
        # print("---")
        return result

    def __lt__(self, other):
        return self.path < other.path

    def __str__(self):
        return self.path.__str__()
    

def greedy_pack(antichains):
    paths = []
    heapq.heapify(antichains)

    while antichains:
        smallest_antichain = heapq.heappop(antichains)
        paths.append(smallest_antichain.path.copy())
        antichains = [antichain for antichain in antichains if smallest_antichain.will_not_break(antichain)]
        heapq.heapify(antichains)

    return paths

# Layered graph setup code

In [92]:
def gen_layered_antichains(num_layers: int):
    antichains = []
    cur_path = []
    def _gen_layered_antichains(cur_layer):
        nonlocal cur_path
        if cur_layer == num_layers:
            antichains.append(Antichain(cur_path.copy()))
            return
        
        cur_layer_start = cur_layer * num_layers
        for node in range(cur_layer_start, cur_layer_start + num_layers):
            cur_path.append(node)
            _gen_layered_antichains(cur_layer + 1)
            cur_path.pop()

    _gen_layered_antichains(0)
    return antichains


# Layered path driver code

In [93]:
graph = gen_layered_antichains(8)
layered_paths = greedy_pack(graph)

In [ ]:
print(len(layered_paths))

25
